## Herramientas para mejorar entrenamiento. 

En esta Notebook entrenaremos diferentes modelos a fin de comparar su perfomance utilizando Grid Search para encontrar los mejores hiperparámetros

Vamos a utilizar como ejemplo un modelo de Aprendizaje Automático Supervisado de Clasificación de Random Forest. Realizaremos los pasos de un Proyecto de Ciencia de Datos:

1. Definición del Problema
2. Búsqueda de datos 
3. Exploración y Limpieza de Datos
4. Dividir los datos en **X** (variables predictoras) e **y** (variable a predecir). Dividir los datos en entrenamiento y testo con el méodo *train_test_split*
5. Entrenamiento del modelo
6. Testeo del Modelo 

### Definición del problema

**¿Qué pasajeros sobrevivirían del hundimiento del Titanic en base a sus características?**

### Búsqueda de datos

El dataset que se utilizará es un listado de los pasajeros del Titanic y se indica si sobrevivieron o no al hundimiento y distintas características de los mismo. Hay varias versiones de este dataset, en este caso utilizaremos uno que tiene realizado una limpieza previa donde se han transformado alguna columnas armando grupos por Edad y dividiendo el dato de grupo familiar ("SibSp_0" y "Parch_0") y ya están realizadas todas las variables dummies. Fue extraído de [Kaggle](https://www.kaggle.com/datasets/ak1352/titanic-cl?select=train_cl.csv). 


In [ ]:
#importamos las librerías que utilizaremos

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
data = pd.read_csv("titanic.csv")

In [ ]:
#vemos los primeros 3 registros
data.head(3)

In [ ]:
#vemos el tamaño del dataset 

data.shape

In [ ]:
data.columns

In [ ]:
# Vemos si hay datos nulos

data.isnull().sum()

In [ ]:
data.info()

In [ ]:
# Vemos la distribución de la variable target

data["Survived"].value_counts()

In [ ]:
# Visualizamos la variable a predecir

sns.countplot(data["Survived"])

#### Random Forest

Ahora entrenaremos distintos modelos de Random Forest modificando los hiperparámetros para intentar lograr mejorar la performance del modelos. Los posibles hiperparámetros que pueden defnirse son los de los Árboles de Decisión y los específicos de los modelos de Random Forest. Pueden verse en la [documentación](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html?highlight=random%20forest#sklearn.ensemble.RandomForestClassifier).

Entre los principales están:

**n_estimators**: Cantidad de árboles a entrenar. El límite de árboles a elegir generalmente viene dado por la capacidad de cómputo en donde se entrena. Sin embargo, también se debe tener en cuenta el riesgo de sobreajuste.

**max_depth**: Máxima profundidad que puede tener cada árbol. La profundidad viene dada por cuantas ramificaciones tiene cada árbol.

**max_features**: Cuantas variables, como máximo, puede tener el conjunto de entrenamiento de cada árbol. Recordar que el conjunto de entrenamiento de cada árbol es un subconjunto del conjunto de entrenamiento completo.

**bootstrap**: Esta es la técnica usada para generar los subconjuntos de datos de entrenamiento para cada árbol. Si se coloca como falso (False) se pasarán todas las filas del data frame al entrenamiento de todos los árboles.

En primer lugar generaremos X e y y dividieremos los datos en entrenamiento y testeo

In [ ]:
X = data.drop(columns="Survived")
y = data["Survived"]

In [ ]:
# Dividimos datos en train y test

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=48)  #por default 25% de test


In [ ]:
print("El tamaño de X_train es: ", X_train.shape)
print("El tamaño de X_test es: ",X_test.shape)
print("El tamaño de y_train es: ",y_train.shape)
print("El tamaño de y_test es: ",y_test.shape)

##### Modelo con los hiperparámetros por default

En primer lugar instanciamos y entrenamos un modelo con los hiperparámetros por default

In [ ]:
# Importamos el modelo que utilizaremos

from sklearn.ensemble import RandomForestClassifier


In [ ]:
# Instanciamos el modelo que utilizaremos con los hiperparametro por default 
random_forest = RandomForestClassifier()

In [ ]:
# Entrenamos el modelo

random_forest.fit(X_train, y_train)

In [ ]:
# Vemos el resultado de la predicción de nuestro modelos sobre los datos utilizados para el entrenamiento

y_pred_train = random_forest.predict(X_train)

In [ ]:
# Utilizamos la métrica accuracy (extactitud)

from sklearn.metrics import accuracy_score

exactitud_train_random_forest= accuracy_score(y_train, y_pred_train)
exactitud_train_random_forest

##### Probamos y evaluamos nuestro modelo

Utilizamos el metodo predict para probar nuestro modelo con los datos de test. Luego comparamos la predicciones de nuestro modelo con el resultado real a través de una matriz de confusión y utilizando la métrica accuracy (exactitud)

In [ ]:
# Probamos nuestro modelo con los datos de test

y_pred_random_forest = random_forest.predict(X_test)
y_pred_random_forest

In [ ]:
# Comparamos y_test(datos reales) con y_pred_arbol (datos predichos) para ver si el modelo hizo las predicciones correctas

from sklearn.metrics import confusion_matrix
matriz_random_forest = confusion_matrix(y_test, y_pred_random_forest)
sns.heatmap(matriz_random_forest, annot=True)
plt.xlabel("Etiquetas predichas")
plt.ylabel("Etiquetas reales")

In [ ]:
# Metrica accuracy

from sklearn.metrics import accuracy_score

exactitud_random_forest = accuracy_score(y_test, y_pred_random_forest)
exactitud_random_forest

#### Grid Search

Es la estrategia para encontrar el mejor hiperparámetro dentro de una
“grilla” (grid) especificada de forma manual. Se realiza una búsqueda
para cada valor de la grilla y se elige el hiperparámetro que
minimiza el error (generalmente calculada
mediante cross-validation -como vimos en la PPT-)

Al utilizar [Grid Search](
https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html?highlight=gridsearch#sklearn.model_selection.GridSearchCV) es necesario definir:

- Método de búsqueda (en este caso Gridsearch - podría ser Randomsearch)
- Estimador: modelo
- Hiperparámetros para hacer la búsqueda.
- Métrica de evaluación


### Probaremos hiperparámetos de Random Forest

Utilizaremos 1 hiperparametro para probar: 

- n_estimators: cantidad de arboles

Para definir la grilla se debe genera un diccionario cuya **clave** va a ser el nombre de los hiperparámetros y el **valor** es una lista que se define de cada hiperparámetro.


https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html?highlight=kn#sklearn.neighbors.KNeighborsClassifier


In [ ]:
#Importamos GridSearch

from sklearn.model_selection import GridSearchCV

In [ ]:
# definimos el estimador/modelo que utilizaremos

random_forest_grid_search = RandomForestClassifier()

In [ ]:
#Definimos la grilla de hiperparámetros para utilizar generando un diccionario:

param_grid = {"n_estimators": [100, 200, 300, 400, 500, 600]}


In [ ]:
# Generamos el grid_search definiendo el modelo, los hiperparametros y la métrica que se utilizará para definir el mejor, en este caso "Accuracy".

grid = GridSearchCV(estimator = random_forest_grid_search, param_grid = param_grid, scoring="accuracy")

In [ ]:
# Entrenamos el modelo de GridSearch con los datos de entrenamiento. Es importante tener en cuenta que puede tardar el entrenamiento

grid.fit(X_train, y_train)

In [ ]:
#Probamos el modelo con los datos de train (VER QUE SE USA GRID)
 
y_pred_train = grid.predict(X_train)
print("La performance del modelo con los datos de entrenamiento:", round(accuracy_score(y_train, y_pred_train),2))

#Probamos el modleo con los datos de test

y_pred = grid.predict(X_test)
print("La performance del modelo con los datos de testo:", round(accuracy_score(y_test, y_pred),2))


A través de los [atributos de GridSearch](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) podemos vemos cuales son las caracerísticas del modelo entrenado, por ejemplo:

- best_params_: mejores hiperparámetros


In [ ]:
grid.best_params_

### Ahora probaremos otros hiperparámetos

Utilizaremos 3 hiperparametros para probar: 

- n_estimators: cantidad de arboles

- max_depth: máxima profundidad del árbol
 
- min_samples_split: mímimo de muestras para hacer una división

Para definir la grilla se debe genera un diccionario cuya **clave** va a ser el nombre de los hiperparámetros y el **valor** es una lista que se define de cada hiperparámetro.


In [ ]:
# Instanciamos el estimador/modelo que utilizaremos
random_forest_grid_search2 = RandomForestClassifier()

In [ ]:
# Definimos los hiperparametros para hacer la búsqueda con un diccionario

param_grid_rf = {"n_estimators": [100,200,300],
                    "max_depth": [2,3,4,5,6],
                    "min_samples_split": [2,3,4,5,6]}

In [ ]:
# Generamos el grid_search definiendo el modelo, los hiperparametros y la métrica que se utilizará para definir el mejor, en este caso "Accuracy".

grid_rf = GridSearchCV(estimator = random_forest_grid_search2, param_grid = param_grid_rf, scoring="accuracy")

In [ ]:
#Entrenamos el modelo. Es importante tener en cuenta que puede tardar el entrenamiento

grid_rf.fit(X_train, y_train)

In [ ]:
#Probamos el modelo con los datos de train (VER QUE SE USA GRID)
 
y_pred_train = grid_rf.predict(X_train)
print("La performance del modelo con los datos de entrenamiento:", round(accuracy_score(y_train, y_pred_train),2))

#Probamos el modleo con los datos de test

y_pred = grid_rf.predict(X_test)
print("La performance del modelo con los datos de testo:", round(accuracy_score(y_test, y_pred),2))


In [ ]:
grid_rf.best_params_

In [ ]:
modelo_random_forest_entrenado = grid_rf.best_estimator_
modelo_random_forest_entrenado

### Conclusión

Grid-Search es una herramienta que se puede utilizar para probar distintos hiperparámetros es un solo entrenamiento lo que puede resultar muy útil para mejorar el rendimiento de los modelos.

Es importante tener en cuenta que la medición del mejor modelo se realiza analizando la performance de los datos de entrenamiento con cross-validation. Luego de seleccionado el mejor modelo es que nosotros/as probamos con los datos de test este modelo para ver su performance.

Este proceso puede implicar que la Notebook tarde mucho tiempo en ejecutarse debido a la gran cantidad de modelos que se están probando (por el cross-validation y por la grilla). 